In [9]:
import io
#import os
#import glob
import imageio
import pandas as pd
import numpy as np
import pathlib as pl
from tqdm import tqdm
from moviepy.editor import *
import matplotlib.pyplot as plt
from IPython.display import Image
from collections import defaultdict

In [10]:
current_path    = pl.Path.cwd().parent.joinpath('img', 'Original')
select_folder   = 1 # 1: train, 2: test
folder_path     = current_path.joinpath('train' if select_folder == 1 else 'test')

images_path     = folder_path.joinpath('ims')
mask_path       = folder_path.joinpath('masks')
images_list     = list(images_path.glob('*.npy'))
mask_list       = list(mask_path.glob('*.npy'))


#create a new folder for the output png images
new_folder=current_path.parent.joinpath('png_images').mkdir(exist_ok=True)
new_folder=current_path.parent.joinpath('png_images').joinpath('train' if select_folder == 1 else 'test').mkdir(exist_ok=True)
output_path=current_path.parent.joinpath('png_images').joinpath('train' if select_folder == 1 else 'test')

#create a new folder for the output gif
new_folder=current_path.parent.joinpath('gif_images').mkdir(exist_ok=True)
new_folder=current_path.parent.joinpath('gif_images').joinpath('train' if select_folder == 1 else 'test').mkdir(exist_ok=True)
output_path_gif=current_path.parent.joinpath('gif_images').joinpath('train' if select_folder == 1 else 'test')


def sorting_key(filepath):
    parts           = filepath.stem.split('_')  # Divide el nombre del archivo en partes
    patient_number  = int(parts[1])  # Extrae el número del paciente
    slice_number    = int(parts[2])  # Extrae el número del slice
    return (patient_number, slice_number)  # Retorna una tupla con ambos números

# Ordena images_list basado en el número del paciente y el número del slice
images_list.sort(key=sorting_key)
mask_list.sort(key=sorting_key)

In [11]:
def generar_imagenes_png(images_list, mask_list, cmap='gray', output_path=output_path):
    """
    Display images and masks side by side.
    
    Parameters:
    - images_list: List of paths to image files.
    - mask_list: List of paths to mask files.
    - cmap: Color map for displaying images and masks. Default is 'gray'.
    
    Returns:
    - None
    """

    for image, mask in tqdm(zip(images_list, mask_list), desc='Displaying images and masks'):
        parts = image.stem.split('_')
        set_folder = parts[0]
        pacient = int(parts[1])
        slice_number = int(parts[2])
        # Cargar datos de imagen y máscara
        image_data = np.load(image)
        mask_data = np.load(mask)
        
        # Crear subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
        
        # Mostrar imagen
        ax1.imshow(image_data, cmap=cmap)
        ax1.set_title(f'Imagen: {image.stem}')
        ax1.axis('off')
        
        # Mostrar máscara
        ax2.imshow(mask_data, cmap=cmap)
        ax2.set_title(f'Máscara: {mask.stem}')
        ax2.axis('off')
        
        # Añadir un súper título con la información del paciente
        fig.suptitle(f'Paciente: {pacient}', fontsize=16)
        
        # Guardar imagen en output_path
        plt.savefig(output_path.joinpath(f'{image.stem}.png'))
        plt.close()

In [12]:
number=20
generar_imagenes_png(images_list[:number], mask_list[:number], cmap='gray', output_path=output_path)

Displaying images and masks: 20it [00:02,  9.22it/s]


In [13]:
def display_images_and_masks(images_list, mask_list, num_slices, counter, pacient, cmap='gray'):
    """
    Display images and masks side by side and store them in a list.
    
    Parameters:
    - images_list: List of paths to image files.
    - mask_list: List of paths to mask files.
    - num_slices: Number of image-mask pairs to display.
    - cmap: Color map for displaying images and masks. Default is 'gray'.
    
    Returns:
    - images_for_gif: List of images for creating a GIF.
    """
    num_slices = num_slices + counter
    
    # Limitar el número de imágenes y máscaras a mostrar
    images_list = images_list[counter:num_slices]
    mask_list = mask_list[counter:num_slices]
    
    # Asegurarse de que hay el mismo número de imágenes y máscaras
    assert len(images_list) == len(mask_list), "La cantidad de imágenes y máscaras debe ser la misma."
    
    images_for_gif = []  # Lista para almacenar imágenes para el GIF
    
    for image, mask in tqdm(zip(images_list, mask_list), desc='Displaying images and masks'):
        # Cargar datos de imagen y máscara
        image_data = np.load(image)
        mask_data = np.load(mask)
        
        # Crear subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
        
        # Mostrar imagen
        ax1.imshow(image_data, cmap=cmap)
        ax1.set_title(f'Imagen: {image.stem}')
        ax1.axis('off')
        
        # Mostrar máscara
        ax2.imshow(mask_data, cmap=cmap)
        ax2.set_title(f'Máscara: {mask.stem}')
        ax2.axis('off')
        
        # Añadir un súper título con la información del paciente
        fig.suptitle(f'Paciente: {pacient:03d}', fontsize=16)
        
        # Guardar la figura en un buffer y luego agregarla a images_for_gif
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        images_for_gif.append(imageio.imread(buf))
        buf.close()
        
        # Cerrar la figura para evitar la visualización
        plt.close(fig)
    
    return images_for_gif

In [14]:
def get_max_slices(images_list):
    slices_per_patient = defaultdict(int)

    for image_path in images_list:
        parts = image_path.stem.split('_')
        patient_number = int(parts[1])
        slice_number = int(parts[2])+1

        # Actualizar el conteo de slices si el número de slice actual es mayor que el máximo registrado
        slices_per_patient[patient_number] = max(slices_per_patient[patient_number], slice_number)

    return slices_per_patient

def print_max_slices(max_slices_per_patient):
    for patient_number, slice_count in max_slices_per_patient.items():
        print(f"Paciente {str(patient_number).zfill(3)}: {slice_count} slices")

# Llamar a la función y obtener el número máximo de slices por paciente
max_slices_per_patient = get_max_slices(images_list)

# Llamar a la función print_max_slices para imprimir la salida formateada
print_max_slices(max_slices_per_patient)

Paciente 000: 49 slices
Paciente 001: 50 slices
Paciente 002: 52 slices
Paciente 003: 29 slices
Paciente 004: 71 slices
Paciente 005: 68 slices
Paciente 006: 60 slices
Paciente 007: 48 slices
Paciente 008: 59 slices
Paciente 009: 66 slices
Paciente 010: 72 slices
Paciente 011: 57 slices
Paciente 012: 76 slices
Paciente 013: 65 slices
Paciente 014: 58 slices
Paciente 015: 55 slices
Paciente 016: 49 slices
Paciente 017: 74 slices
Paciente 018: 68 slices
Paciente 019: 60 slices
Paciente 020: 45 slices
Paciente 021: 75 slices
Paciente 022: 48 slices
Paciente 023: 62 slices
Paciente 024: 43 slices
Paciente 025: 66 slices
Paciente 026: 46 slices
Paciente 027: 45 slices
Paciente 028: 50 slices
Paciente 029: 52 slices
Paciente 030: 70 slices
Paciente 032: 46 slices
Paciente 033: 66 slices
Paciente 034: 28 slices
Paciente 035: 57 slices
Paciente 036: 14 slices
Paciente 037: 45 slices
Paciente 038: 25 slices
Paciente 039: 72 slices
Paciente 040: 77 slices
Paciente 041: 72 slices
Paciente 043: 66

In [15]:
def create_dataframe(max_slices_per_patient):
    # Convertir el diccionario a una lista de tuples
    data_list = [(str(patient_number).zfill(3), slice_count) for patient_number, slice_count in max_slices_per_patient.items()]
    
    # Crear un DataFrame y retornarlo
    return pd.DataFrame(data_list, columns=['Paciente', 'max_slices'])

df = create_dataframe(max_slices_per_patient)

df['max_slices'].describe()

count    48.000000
mean     56.770833
std      15.315462
min      14.000000
25%      47.750000
50%      58.500000
75%      69.000000
max      85.000000
Name: max_slices, dtype: float64

In [16]:
df

,Paciente,max_slices
0,000,49
1,001,50
2,002,52
3,003,29
4,004,71
5,005,68
6,006,60
7,007,48
8,008,59
9,009,66


In [17]:
counter = 0
for patient_number, max_slices in tqdm(max_slices_per_patient.items(), desc='Processing patients'):
    #print(f'Paciente {patient_number:03d}: {max_slices} slices')
    images_for_gif = display_images_and_masks(images_list, mask_list, max_slices, counter,patient_number,cmap='gray')
    counter = counter + max_slices + 1
    if images_for_gif:
        # Guardar el GIF en output_path_gif
        imageio.mimsave(output_path_gif.joinpath(f'paciente_{patient_number:03d}.gif'), images_for_gif, duration=0.5)
    else:
        print(f'No se generaron imágenes para el paciente {patient_number:03d}')


Displaying images and masks: 49it [00:06,  8.05it/s] ?it/s]
Displaying images and masks: 50it [00:06,  7.81it/s]:11,  6.62s/it]
Displaying images and masks: 52it [00:06,  7.70it/s]:10,  6.76s/it]
Displaying images and masks: 29it [00:03,  7.44it/s]:13,  6.97s/it]
Displaying images and masks: 71it [00:09,  7.78it/s]:18,  5.87s/it]
Displaying images and masks: 68it [00:08,  7.76it/s]:12,  7.27s/it]
Displaying images and masks: 60it [00:08,  7.39it/s]:35,  8.00s/it]
Displaying images and masks: 48it [00:05,  8.31it/s]:36,  8.21s/it]
Displaying images and masks: 59it [00:08,  7.23it/s]:02,  7.57s/it]
Displaying images and masks: 66it [00:07,  8.26it/s]:09,  7.94s/it]
Displaying images and masks: 72it [00:10,  7.19it/s]5:09,  8.14s/it]
Displaying images and masks: 57it [00:06,  8.18it/s]5:30,  8.92s/it]
Displaying images and masks: 76it [00:09,  8.33it/s]5:05,  8.49s/it]
Displaying images and masks: 65it [00:09,  6.83it/s]5:11,  8.89s/it]
Displaying images and masks: 58it [00:07,  8.25it/s]

No se generaron imágenes para el paciente 049


In [19]:
#create a new folder for the output gif
new_folder      = current_path.parent.joinpath('mp4_videos').mkdir(exist_ok=True)
new_folder      = current_path.parent.joinpath('mp4_videos').joinpath('train' if select_folder == 1 else 'test').mkdir(exist_ok=True)
output_path_mp4 = current_path.parent.joinpath('mp4_videos').joinpath('train' if select_folder == 1 else 'test')

In [21]:
gif_list = list(current_path.glob('*.gif'))
for gif in gif_list:
    display(Image(str(gif)))

In [22]:
def convert_gif_to_mp4(gif_path, mp4_path):
    clip = VideoFileClip(str(gif_path))
    clip.write_videofile(str(mp4_path), codec='libx264')

In [23]:
gif_list = list(output_path_gif.glob('*.gif'))
for gif_path in gif_list:
    mp4_filename = output_path_mp4.joinpath(f"{gif_path.stem}.mp4")
    convert_gif_to_mp4(gif_path, mp4_filename)

Moviepy - Building video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_000.mp4.
Moviepy - Writing video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_000.mp4



Moviepy - Done !
Moviepy - video ready d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_000.mp4
Moviepy - Building video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_001.mp4.
Moviepy - Writing video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_001.mp4



Moviepy - Done !
Moviepy - video ready d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_001.mp4
Moviepy - Building video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_002.mp4.
Moviepy - Writing video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_002.mp4



Moviepy - Done !
Moviepy - video ready d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_002.mp4
Moviepy - Building video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_003.mp4.
Moviepy - Writing video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_003.mp4



Moviepy - Done !
Moviepy - video ready d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_003.mp4
Moviepy - Building video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_004.mp4.
Moviepy - Writing video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_004.mp4



...

Moviepy - Done !
Moviepy - video ready d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_044.mp4
Moviepy - Building video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_045.mp4.
Moviepy - Writing video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_045.mp4



Moviepy - Done !
Moviepy - video ready d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_045.mp4
Moviepy - Building video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_046.mp4.
Moviepy - Writing video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_046.mp4



Moviepy - Done !
Moviepy - video ready d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_046.mp4
Moviepy - Building video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_047.mp4.
Moviepy - Writing video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_047.mp4



Moviepy - Done !
Moviepy - video ready d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_047.mp4
Moviepy - Building video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_048.mp4.
Moviepy - Writing video d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_048.mp4



Moviepy - Done !
Moviepy - video ready d:\Trabajo de Maestria\segmentacion_maestria\img\mp4_videos\train\paciente_048.mp4
